First of all libraries needed

In [1]:
import numpy as np
import matplotlib.pyplot as plt

First off all we are going to generate a random data sample for a regression model, considering the contraints proposed in the problem:

 - 0 $\leq \beta_{i} \leq 5 \hspace{0.5cm}$ i = 0,...,K for K at least 1000 independent variables and n=5000 observations

In [2]:
import numpy as np

K=1000

n_train=50000
n_test=2000
n=n_train+n_test
nvars=1000

randombeta = np.random.randint(0,5,size=([nvars+1,1]))
randomerror = np.random.normal(0,1,(n,1))

X0 = np.ones([n,1]) # the first column has all values equal to one for the coefficients of beta_0
X1 = np.random.uniform(0,10,([n,nvars]))
randomX = np.concatenate([X0, X1],axis=1)

y = np.dot(randomX,randombeta) + randomerror

X = randomX[0:n_train,:]
Y = y[0:n_train]

X_test = randomX[(n_train+1):n,:]
Y_test = y[(n_train+1):n]



a) (0.5 points) Estimate the value of the regression coefficients by implementing the analytical
solution. Use this solution as a benchmark for the following sections.

We have to minimize:

\begin{align*}
  \text{minimize}_\beta \quad & \|y-X\beta\|^2_2 + \rho\|\beta\|^2_2
\end{align*}

Then, we can rewrite the formula as:

\begin{align*}
  (y-X\beta)^T(y-X\beta) + \rho(\beta^T\beta)
\end{align*}

\begin{align*}
  Y^TY - Y^TX\beta - \beta^TX^TY + \beta^TX^T\beta X + \rho (\beta^T \beta)
\end{align*}

Then, for minimizing we have to take the partial derivative and level it to 0.

\begin{align*}
  \frac{\partial F}{\partial\beta} = 0  
\end{align*}

\begin{align*}
  \frac{\partial F}{\partial\beta} = -2X^TY + 2X^TX\beta + \rho \beta  
\end{align*}

\begin{align*}
  -2X^TY + 2X^TX\beta + \rho \beta = 0
\end{align*}

 
Now isolating the variable $\beta$ to one of the sides we obtain the exact optimal solution of the preceding problem, which is: 

\begin{align*}
  \beta_{ls}=(X^T X + p/2)^{-1}X^T Y
\end{align*}

In [3]:

from numpy.linalg import inv
import time


time_start = time.process_time()

## Apply the formula

beta_ls_exact = np.dot(np.dot(inv(np.dot((X.T),X)+(5/2)),X.T),Y)
time_elapsed = (time.process_time() - time_start)

## Print the results

print('Values of the (exact) least squares coefficients:')
for i in range(nvars+1):
    print('beta %3d %7.3f' %(i,beta_ls_exact[i]))
print('Elapsed time = %8.5f' %(time_elapsed))


Values of the (exact) least squares coefficients:
beta   0 -218.826
beta   1   3.048
beta   2   4.038
beta   3   2.043
beta   4   2.052
beta   5   2.049
beta   6   3.057
beta   7   4.042
beta   8   3.054
beta   9   4.042
beta  10   2.047
beta  11   4.040
beta  12   3.044
beta  13   0.036
beta  14   3.045
beta  15   0.047
beta  16   2.044
beta  17   0.047
beta  18   1.053
beta  19   4.045
beta  20   1.048
beta  21   2.040
beta  22   2.040
beta  23   0.043
beta  24   0.040
beta  25   0.044
beta  26   0.050
beta  27   2.057
beta  28   4.046
beta  29   1.039
beta  30   0.052
beta  31   4.043
beta  32   3.042
beta  33   2.038
beta  34   2.045
beta  35   4.042
beta  36   3.048
beta  37   1.059
beta  38   0.044
beta  39   2.035
beta  40   1.066
beta  41   4.036
beta  42   4.032
beta  43   0.053
beta  44   0.053
beta  45   3.039
beta  46   3.044
beta  47   4.039
beta  48   0.042
beta  49   2.032
beta  50   1.060
beta  51   0.036
beta  52   0.036
beta  53   2.036
beta  54   2.033
beta  55   1.0


b) (1 points) Estimate the value of the regression coefficients by using the function minimize from the Python module Scipy.optimize. Try at least four available solvers and compare their performance in terms of number of iterations, number of function, gradient and hessian evaluations as well as total computational time.

In [20]:
from scipy.optimize import minimize

# Define the objective function as the sum of squares of the errors

def least_sq_ridreg(beta_ls, X, Y,p):
    beta_ls = np.matrix(beta_ls)
    z = Y - np.dot(X,beta_ls.T)
    a=np.dot(z.T,z)
    b=np.dot(beta_ls,beta_ls.T)
    return (a+p*b)

def least_sq_ridreg_der(beta_ls,X,Y,p):
    beta_ls = np.matrix(beta_ls)
    pp = -2*np.dot((Y-np.dot(X,(beta_ls).T)).T,X) + p*beta_ls
    aa = np.squeeze(np.asarray(pp))
    return aa

def least_sq_ridreg_hess(beta_ls,X,Y,p):
    ss = 2*np.dot(np.transpose(X),X) + p
    return ss

beta_ls0 = np.zeros(nvars+1)

#time_start_Nelder = time.process_time()
## Solve the optimization algorithm using a derivative-free method
#res = minimize(least_sq_ridreg, beta_ls0, args=(X, Y,5), method='Nelder-Mead', options={'disp': True,'xtol': 1e-10})
#time_elapsed_Nelder = (time.process_time() - time_start_Nelder)

We are now going to include the gradient and hessian evaluation

In [ ]:
time_start = time.process_time()

res = minimize(least_sq_ridreg, beta_ls0, args=(X, Y), method='Newton-CG', jac=least_sq_ridreg_der, hess=least_sq_ridreg_hess, options={'disp': True})

time_elapsed = (time.process_time() - time_start) 

Now with the Powel method

In [ ]:
time_start_Powell = time.process_time()
## Solve the optimization algorithm using a derivative-free method
res = minimize(least_sq_ridreg, beta_ls0, args=(X, Y,5), method='Powell', options={'disp': True,'xtol': 1e-10})
time_elapsed_Powell = (time.process_time() - time_start_Powell)

Now using CG

In [ ]:
time_start_CG = time.process_time()
## Solve the optimization algorithm using a derivative-free method
res = minimize(least_sq_ridreg, beta_ls0, args=(X, Y,5), method='CG', options={'disp': True,'xtol': 1e-10})
time_elapsed_CG = (time.process_time() - time_start_CG)

Finally TNC method

In [ ]:
time_start_TNC = time.process_time()
## Solve the optimization algorithm using a derivative-free method
res = minimize(least_sq_ridreg, beta_ls0, args=(X, Y,5), method='TNC', options={'disp': True,'xtol': 1e-10})
time_elapsed_TNC = (time.process_time() - time_start_TNC)


c) (1 points) Modify the preceding optimization model by adding (lower and upper) bounds on the values of the β coefficients. Solve it again with the module Scipy.optimize a by using (at least) two different solvers, which should accept the introduction of bounds on the variables. Compare these methods and briefly comment on possible interpretations of the values of the coefficients.

In [23]:
time_start_TNC_bounds = time.process_time()
## Solve the optimization algorithm using a derivative-free method
#res = minimize(least_sq_ridreg, beta_ls0, args=(X, Y,5), method='TNC', bounds=(((0,5),)*len(beta_ls0)),options={'disp': True,'xtol': 1e-10})
res = minimize(least_sq_ridreg, beta_ls0, args=(X, Y,5), method='TNC', bounds=(((0,5),)*len(beta_ls0)), jac=least_sq_ridreg_der, hess=least_sq_ridreg_hess, options={'disp': True})
time_elapsed_TNC_bounds = (time.process_time() - time_start_TNC_bounds)

## Print results
print('\nValues of the least squares coefficients obtained with SLSQP for the Lasso problem:')
for i in range(nvars+1):
    print('beta %3d %7.3f' %(i,res.x[i]))
print('Elapsed time = %8.5f' %(time_elapsed))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/scipy/optimize/_minimize.py:547: RuntimeWarning: Method TNC does not use Hessian information (hess).
  warn('Method %s does not use Hessian information (hess).' % method,
  NIT   NF   F                       GTG
    0    1  4.815550828274299E+12   2.40804087E+22
tnc: fscale = 1.28884e-12
tnc: stepmx = 1000
    1    4  1.824990663618377E+11   9.08627630E+20
    2    7  1.104509736831472E+10   5.48607550E+19
tnc: fscale = 2.70022e-11
    3   10  2.155631371053159E+09   1.06925298E+19
    4   13  2.055017710887169E+09   1.01831807E+19
    5   16  1.924355682913226E+09   9.52608909E+18
    6   19  1.534455689940606E+09   7.58811609E+18
    7   22  1.075971420687867E+09   5.31514506E+18
    8   25  8.469516651867350E+08   4.17940157E+18
    9   28  4.357937770597357E+08   2.14797941E+18
   10   31  3.829950474217591E+08   1.88579781E+18
   11   34  1.698047444665177E+08   8.34960655E+17
   12   37  8.16538800953


Values of the least squares coefficients obtained with SLSQP for the Lasso problem:
beta   0   0.491
beta   1   2.998
beta   2   4.001
beta   3   1.998
beta   4   2.002
beta   5   2.002
beta   6   2.999
beta   7   4.002
beta   8   3.000
beta   9   4.001
beta  10   2.000
beta  11   3.997
beta  12   3.000
beta  13   0.002
beta  14   3.001
beta  15   0.000
beta  16   2.000
beta  17   0.000
beta  18   0.999
beta  19   3.997
beta  20   1.000
beta  21   1.997
beta  22   2.002
beta  23   0.001
beta  24   0.002
beta  25   0.000
beta  26   0.003
beta  27   2.000
beta  28   4.001
beta  29   1.002
beta  30   0.000
beta  31   3.998
beta  32   3.000
beta  33   2.000
beta  34   2.001
beta  35   3.998
beta  36   2.997
beta  37   1.001
beta  38   0.000
beta  39   2.001
beta  40   1.002
beta  41   3.999
beta  42   4.001
beta  43   0.000
beta  44   0.001
beta  45   2.999
beta  46   3.001
beta  47   4.001
beta  48   0.001
beta  49   1.997
beta  50   1.000
beta  51   0.000
beta  52   0.000
beta  53   2.0

  127  485  7.805628453869485E+04   1.30630107E+04
tnc: Linear search failed


In [27]:
time_start_BFGS = time.process_time()
## Solve the optimization algorithm using a derivative-free method
#res = minimize(least_sq_ridreg, beta_ls0, args=(X, Y,5), method='TNC', bounds=(((0,5),)*len(beta_ls0)),options={'disp': True,'xtol': 1e-10})
res = minimize(least_sq_ridreg, beta_ls0, args=(X, Y,5), method='trust-krylov', bounds=(((0,5),)*len(beta_ls0)), jac=least_sq_ridreg_der, hess=least_sq_ridreg_hess, options={'disp': True})
time_elapsed_BFGS = (time.process_time() - time_start_BFGS)

## Print results
print('\nValues of the least squares coefficients obtained with SLSQP for the Lasso problem:')
for i in range(nvars+1):
    print('beta %3d %7.3f' %(i,res.x[i]))
print('Elapsed time = %8.5f' %(time_elapsed))

 TR Solving trust region problem, radius 1.000000e+00; starting on first irreducible block
 TR Coldstart. Seeking suitable initial Î»â, starting with 0
 TR Starting Newton iteration for Î»â with initial choice 0.000000e+00
 TR  iter        Î»            dÎ»       âhâ(Î»)â-radius
 TR      1  1.526779e+11  1.526779e+11  0.000000e+00


 iter inewton type    objective     Î³áµ¢ââ|háµ¢|      leftmost         Î»             Î³             Î´             Î±             Î²       

     0     1  cg_b -1.539283e+11  6.022927e+05  0.000000e+00  1.526779e+11  1.551786e+11  2.500709e+09  3.998866e-10  5.800813e-08


 TR Solving trust region problem, radius 2.000000e+00; starting on first irreducible block
 TR Coldstart. Seeking suitable initial Î»â, starting with 0
 TR Starting Newton iteration for Î»â with initial choice 0.000000e+00
 TR  iter        Î»            dÎ»       âhâ(Î»)â-radius
 TR      1  7.383826e+10  7.383826e+10  0.000000e+00


 iter inewton type    objectiv